# Adding a new OMOP CDM dataset

Follow the steps in this notebook to add a new OMOP CDM dataset and make it available through ATLAS

In [ ]:
import sqlalchemy as sa
import getpass

## Settings

Replace these parameters

In [ ]:
CDM_SCHEMA = ''  # 'erspc'
RESULTS_SCHEMA = ''  # 'erspc_results'
CDM_DATA_PATH = ''  # '../data/erspc_omop_cdm.sql'

# Connect to the db

In [ ]:
server = 'postgresql'
port = '5432'
db = 'ohdsi'
engine = sa.create_engine(f'postgresql://{input("User:")}:{getpass.getpass("Password:")}@{server}:{port}/{db}')

In [ ]:
#sa.inspect(engine).get_schema_names()

In [ ]:
#sa.inspect(engine).get_table_names('erspc_results')

In [ ]:
if CDM_SCHEMA in sa.inspect(engine).get_schema_names():
    print('WARNING: the schema already exists. Proceed with care')

## Create CDM

In [ ]:
con = engine.connect()

In [ ]:
# con.execute('DROP SCHEMA %s CASCADE;' % CDM_SCHEMA)
con.execute('CREATE SCHEMA %s;' % CDM_SCHEMA)

In [ ]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open('pioneer_omop_cdm/OMOP CDM postgresql v6_0_onco_modified ddl.sql') as f_ddl:
    con.execute(f_ddl.read())

## Load data

Manual step: bulk load the delimited text files into the corresponding OMOP tables.

## Apply constraints, indexes and add vocab views

In [ ]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open('pioneer_omop_cdm/OMOP CDM postgresql v6_0_dev pk indexes.sql') as f_ind:
    con.execute(f_ind.read())

In [ ]:
with open('pioneer_omop_cdm/OMOP CDM postgresql v6_0_dev constraints.sql') as f_constr:
    con.execute(f_constr.read())

In [ ]:
con.execute('SET search_path TO %s;' % CDM_SCHEMA)
with open('other/vocab_view.sql') as f_vocab_view:
    con.execute(f_vocab_view.read())

## Setup results schema

In [ ]:
con = engine.connect()

In [ ]:
# con.execute('DROP SCHEMA %s CASCADE;' % RESULTS_SCHEMA)
con.execute('CREATE SCHEMA %s;' % RESULTS_SCHEMA)

In [ ]:
con.execute('SET search_path TO %s;' % RESULTS_SCHEMA)
with open('other/results_ddl_2.9.0.sql') as f_results:
    con.execute(sa.text(f_results.read()).execution_options(auto_commit=True))

In [ ]:
con.execute('CREATE OR REPLACE VIEW concept_hierarchy AS (SELECT * FROM vocab.concept_hierarchy);')

In [ ]:
con.close()

## Run Achilles (R)

Run Achilles R script

## Source Daimon

_manual step_

To add in Atlas->Configuration

Note: had to 'Clear Configuration Cash' to make it work. Gettign EntityExsistsErrors before that

========================================================